# Importações

In [311]:
from faker import Faker
import random
import time
from datetime import datetime
import sys
import os
import re
from requests import post
from bs4 import BeautifulSoup
import pandas as pd
from geopy.geocoders import ArcGIS
from geopy.extra.rate_limiter import RateLimiter

In [312]:
DEBUG = False

# Função para adicionar o caminho do módulo 'utils' ao sys.path
utils_path = os.path.abspath("../ETL/utils/") if DEBUG else  os.path.abspath("../")

if utils_path not in sys.path:
    sys.path.append(utils_path)

In [313]:
from utils.auxiliary_functions.all_auxiliary_functions import (
    extrair_dados_database,
    carregar_dados_database,
)

## Funções Gerais

In [314]:
# Dicionário para armazenar os DREs gerados por ano e semestre sem repetição
DRE_REGISTRO = {}

def gerar_dre(periodo_ingresso_ufrj):
    periodo, semestre = periodo_ingresso_ufrj.split('/')
    periodo = periodo[-2:]
    semestre = str(int(semestre)-1)
    
    chave = f'{periodo}{semestre}'
    if chave not in DRE_REGISTRO:
        DRE_REGISTRO[chave] = set()
    
    while True:
        ultimos_digitos = f'{random.randint(0, 99999):05d}'
        dre = f'1{periodo}{semestre}{ultimos_digitos}'
        if ultimos_digitos not in DRE_REGISTRO[chave]:
            DRE_REGISTRO[chave].add(ultimos_digitos)
            return dre

In [315]:
def gerar_cep(estado="", cidade="", tentativas=5):
    for tentativa in range(tentativas):
        try:
            # Faz a requisição para gerar o CEP
            r = post(
                "https://www.4devs.com.br/ferramentas_online.php",
                data={
                    "acao": "gerar_cep",
                    "cep_estado": estado,  # Pode ser preenchido com um estado específico ou deixado vazio para aleatório
                    "cep_cidade": cidade,  # Pode ser preenchido com uma cidade específica ou deixado vazio para aleatório
                    "somente_numeros": "n",  # 'n' para gerar CEPs formatados com pontuação, 's' para só números
                },
            )

             # Verifica se a requisição foi bem-sucedida
            if r.status_code == 200:
                soup = BeautifulSoup(r.text, "html.parser")
                
                # Extrair informações dentro dos <span>
                cep = soup.find("div", id="cep").span.text
                endereco = soup.find("div", id="endereco").span.text
                bairro = soup.find("div", id="bairro").span.text
                cidade = soup.find("div", id="cidade").span.text
                estado = soup.find("div", id="estado").span.text
                
                # Retorna os dados coletados
                return {
                    "CEP": cep,
                    "Logradouro": endereco,
                    "Bairro": bairro,
                    "Cidade": cidade,
                    "Estado": estado,
                }
            else:
                print(f"Erro: {r.status_code}")
        
        except Exception as e:
            # print(f"Erro na requisição: {e}")
            pass
        
        # Implementa o exponencial backoff para retries
        tempo_espera = 2 ** tentativa  # O tempo de espera aumenta exponencialmente
        # print(f"Tentativa {tentativa + 1} falhou, esperando {tempo_espera} segundos antes de tentar novamente.")
        time.sleep(tempo_espera)
    
    # Se todas as tentativas falharem, retorna valores nulos
    return {
        "CEP": None,
        "Logradouro": None,
        "Bairro": None,
        "Cidade": None,
        "Estado": None
    }

In [316]:
def aplicar_gerar_cep(row, total_requisicao):
    if total_requisicao % 30 == 0:
        print(f"Total de requisições: {total_requisicao}")
        time.sleep(60)
    
    dados_cep = gerar_cep(estado='RJ', cidade=7043)  # 7043 é o código da cidade do RJ
    return pd.Series(dados_cep)

In [317]:
# Função para aplicar o join ou copiar as colunas caso as colunas necessárias não existam
def processar_enderecos(df_bronze, df_enderecos):
    # Verificar se as colunas ['CEP', 'Logradouro', 'Bairro', 'Cidade', 'Estado'] existem em df_bronze
    required_columns = ['CEP', 'Logradouro', 'Bairro', 'Cidade', 'Estado']
    
    if all(col in df_bronze.columns for col in required_columns):
        # Faz o join entre os dados dos alunos e a tabela local de endereços
        df_com_enderecos = df_bronze.merge(
            df_enderecos, 
            on=['Bairro', 'Cidade', 'Estado', 'CEP', 'Logradouro'], 
            how='left',  # Join à esquerda, para que alunos sem dados no banco local apareçam com NaN
            suffixes=('', '_local')
        )
    else:
        df_com_enderecos = df_bronze.copy()
        # Caso as colunas não existam, apenas copiar e colar as colunas de endereços e preencher NaN
        df_com_enderecos[required_columns] = df_enderecos[required_columns]
    
    # Filtrar alunos que ainda não têm CEP, ou seja, que não foram encontrados no banco local
    df_sem_cep = df_com_enderecos[df_com_enderecos['CEP'].isna()]
    
    return df_com_enderecos, df_sem_cep

In [318]:
def calcular_idade_ingresso_curso_atual(row):
    """
    Calcula a idade atual com base na data de nascimento fornecida.
    
    Args:
    data_nascimento (str): Data de nascimento no formato 'dd/mm/yyyy'.
    
    Returns:
    int: Idade atual em anos, ou None se a data estiver em um formato inválido.
    """
    data_nascimento = row['dataNascimento']
    periodo_ingresso = row['periodoIngressoCursoAtual']
    try:
        # Converter a string da data de nascimento em um objeto datetime
        ano_nascimento = datetime.strptime(data_nascimento, '%d/%m/%Y').year

        # Obter a data atual
        ano_ingresso = int(periodo_ingresso.split('/')[0])

        # Calcular a idade
        idade = ano_ingresso - ano_nascimento
        return idade
    except ValueError:
        # Retornar None se a data de nascimento estiver em um formato inválido
        return None

In [319]:
# Lista de títulos a serem removidos
TITULOS = ["Sr.", "Sra.", "Dr.", "Dra."]
FAKER = Faker("pt_BR")

# Função para remover os títulos do nome
def remover_titulos(nome):
    # Cria um padrão regex que procura os títulos no início do nome
    padrao = r'^(?:' + '|'.join(re.escape(titulo) for titulo in TITULOS) + r')\s*'
    # Substitui os títulos encontrados por uma string vazia
    return re.sub(padrao, '', nome).strip()

# Função para gerar nomes de acordo com o sexo
def gerar_nome_por_sexo(sexo):
    if sexo.lower() == "m":
        nome = FAKER.name_male()
    elif sexo.lower() == "f":
        nome = FAKER.name_female()
    else:
        nome = FAKER.name()  # Gera um nome genérico em caso de valor desconhecido
    
    # Remove títulos do nome gerado
    return remover_titulos(nome)

In [320]:
GEOLOCATOR = ArcGIS(user_agent='myGeoCoder')
GEOCODE = RateLimiter(GEOLOCATOR.geocode, min_delay_seconds=1)

# Função para obter latitude e longitude, verificando antes no df_coord_enderecos e adicionando novas coordenadas
def obter_lat_lon(row, df_coord_enderecos):
    # Verifica se o endereço já está presente no df_coord_enderecos
    result = df_coord_enderecos[
        (df_coord_enderecos['Bairro'] == row['Bairro']) & 
        (df_coord_enderecos['Estado'] == row['Estado'])
    ]

    if not result.empty:
        # Se o endereço já existe no df_coord_enderecos, usa suas coordenadas
        return (result.iloc[0]['Latitude'], result.iloc[0]['Longitude']), df_coord_enderecos
    else:
        # Se não estiver no df_coord_enderecos, faz a geocodificação
        location = GEOCODE(f"{row['Bairro']}, {row['Estado']}")
        if location:
            lat_lon = (location.latitude, location.longitude)
        else:
            lat_lon = (None, None), df_coord_enderecos
        
        # Adiciona as novas coordenadas ao df_coord_enderecos
        novo_registro = pd.DataFrame({
            'Bairro': [row['Bairro']],
            'Estado': [row['Estado']],
            'Latitude': [lat_lon[0]],
            'Longitude': [lat_lon[1]]
        })
        # Concatena o novo registro ao DataFrame original
        df_coord_enderecos = pd.concat([df_coord_enderecos, novo_registro], ignore_index=True)

        return lat_lon, df_coord_enderecos

# Processamento


In [321]:
df_bronze = extrair_dados_database("alunos/alunos.parquet", "bronze")

## Criação de Colunas com Dados Ficticios

**Descrição da Etapa**

Nesta etapa, serão gerados os dados referentes ao nome do aluno, com base no seu gênero, a matrícula (DRE), conforme o período de ingresso, e os respectivos endereços dos alunos.

In [323]:
df_bronze["nomeCompleto"] = df_bronze["sexo"].apply(gerar_nome_por_sexo)
df_bronze["matriculaDre"] = df_bronze['periodoIngressoUFRJ'].apply(gerar_dre)

In [325]:
try:
    # Tentar extrair o banco de endereços local
    enderecos = extrair_dados_database("enderecos/enderecos.parquet", "silver")
    df_com_cep, df_sem_cep = processar_enderecos(df_bronze, enderecos)

    # Se houver registros sem CEP, fazer a requisição à API
    if not df_sem_cep.empty:
        total_requisicao = 1
        for idx, row in df_sem_cep.iterrows():
            # Atualizar a linha com o resultado da API
            dados_cep = aplicar_gerar_cep(row, total_requisicao)
            df_com_cep.loc[idx, ["CEP", "Logradouro", "Bairro", "Cidade", "Estado"]] = (
                dados_cep
            )
            total_requisicao += 1

        # Após completar os dados com a API, adicionar novos endereços ao banco local
        novos_enderecos = df_com_cep[
            ["Bairro", "Cidade", "Estado", "CEP", "Logradouro"]
        ].dropna()
        df_bronze[["Bairro", "Cidade", "Estado", "CEP", "Logradouro"]] = (
            novos_enderecos[["Bairro", "Cidade", "Estado", "CEP", "Logradouro"]]
        )
        # Atualizar o banco local com os novos endereços
        carregar_dados_database(
            novos_enderecos, "enderecos/enderecos", "silver", "parquet"
        )
    else:
        df_bronze = df_com_cep.copy()

except FileNotFoundError:
    # Se o banco local não existe, processar todos os endereços via API
    for idx, row in df_bronze.iterrows():
        df_bronze.loc[idx, ["CEP", "Logradouro", "Bairro", "Cidade", "Estado"]] = (
            aplicar_gerar_cep(row, idx + 1)
        )

    # Salvar os endereços processados para uso futuro
    enderecos = (
        df_bronze[["Bairro", "Cidade", "Estado", "CEP", "Logradouro"]].dropna().copy()
    )
    carregar_dados_database(enderecos, "enderecos/enderecos", "silver", "parquet")

In [327]:
# Extraindo tabela com as coordenadas dos bairros para ser usada como Cache.
df_coord_enderecos = extrair_dados_database("enderecos/coord_enderecos.parquet", "silver")

# Aplica a função ao DataFrame df_bairros e cria as novas colunas 'Latitude' e 'Longitude'
for idx, row in df_bronze.iterrows():
    lat_lon, df_coord_enderecos = obter_lat_lon(row, df_coord_enderecos)
    df_bronze.at[idx, 'Latitude'] = lat_lon[0]
    df_bronze.at[idx, 'Longitude'] = lat_lon[1]

In [ ]:
# Carrega o DataFrame atualizado no database
carregar_dados_database(df_coord_enderecos, "enderecos/coord_enderecos", "silver", "parquet")

### Porcessamento normal

In [329]:
# Calcula a idade de quando o aluno ingressou no curso Atual
df_bronze['idadeIngressoCursoAtual'] = df_bronze.apply(calcular_idade_ingresso_curso_atual, axis=1)

In [331]:
# Filtra somente por alunos que estao no curso da CC
lista_cod_curso_computacao = [3101020000, 3101070000, 3109000100]
df_bronze = df_bronze[df_bronze["codCursoAtual"].isin(lista_cod_curso_computacao)]

In [261]:
# Normalizando a coluna sexo
mapeamento_genero = {
    'M': 'Masculino',
    'F': 'Feminino'
}
df_bronze['sexo'] = df_bronze['sexo'].map(mapeamento_genero)

In [ ]:
# Salvar dados na camada Silver
carregar_dados_database(df_bronze, "alunos/alunos", "silver", "parquet")